# Interpreting Identity

# Setup

In [1]:
TRAIN_MODEL = True

In [2]:
DEVELOPMENT_MODE = True
# Upgrade pip
%pip install --upgrade pip

from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")
ipython.run_line_magic("pip", "install ipympl")
ipython.run_line_magic("pip", "install scipy")
ipython.run_line_magic("pip", "install manim")
ipython.run_line_magic("pip", "install torch")
ipython.run_line_magic("pip", "install numpy<2")
ipython.run_line_magic("pip", "install einops")
ipython.run_line_magic("pip", "install transformer_lens")

Note: you may need to restart the kernel to use updated packages.


/tmp/ipykernel_652075/1990858556.py:9: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_652075/1990858556.py:10: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
/bin/bash: Zeile 1: 2: Datei oder Verzeichnis nicht gefunden
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import stuff
import torch
import numpy as np
import os
import tqdm.auto as tqdm
from pathlib import Path

import copy

from transformer_lens import HookedTransformer, HookedTransformerConfig, ActivationCache

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# Define the location to save the model, using a relative path
PTH_LOCATION = "workspace/_scratch/identity.pth"

# Create the directory if it does not exist
os.makedirs(Path(PTH_LOCATION).parent, exist_ok=True)

cpu


/home/pweiss/Research/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


# Model Training

## Config

In [4]:
INPUT_DIM = 3
OUTPUT_DIM = 3
frac_train = 1

# Optimizer config
lr = 1e-3
wd = 1e-2
betas = (0.9, 0.999)

num_epochs = 10000
checkpoint_every = 500

DATA_SEED = 599

## Define Task
* Define random function
* Define the dataset & labels

Input format:
|a|b|

In [5]:
def get_training_data(input_dim = INPUT_DIM, output_dim = OUTPUT_DIM, data_seed = DATA_SEED):
    torch.manual_seed(data_seed)
    a_vector = torch.arange(input_dim)
    dataset = torch.cartesian_prod(a_vector, a_vector).to(device)

    labels = torch.randint(0, output_dim, (dataset.shape[0],), device=device)
    train_data = dataset
    train_labels = labels
    # For now no test data
    test_data = dataset[0:0]
    test_labels = labels[0:0]
    print(train_data)
    print(train_labels)
    print(train_data.shape)
    print(test_data[:5])
    print(test_labels[:5])
    print(test_data.shape)
    return train_data, train_labels, test_data, test_labels

train_data, train_labels, test_data, test_labels = get_training_data()

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2]])
tensor([0, 1, 2, 1, 1, 0, 2, 2, 0])
torch.Size([9, 2])
tensor([], size=(0, 2), dtype=torch.int64)
tensor([], dtype=torch.int64)
torch.Size([0, 2])


## Define Model

In [6]:

def get_seeded_model(seed = 999, input_dim = INPUT_DIM, output_dim = OUTPUT_DIM):
    cfg = HookedTransformerConfig(
        n_layers = 1,
        n_heads = 1,
        d_model = 2,
        d_head = 2,
        d_mlp = 1,
        attn_only=False,
        act_fn = "relu",
        normalization_type=None,
        d_vocab=input_dim,
        d_vocab_out=output_dim,
        n_ctx=2,
        init_weights=True,
        device=device,
        seed = seed,
    )
    model = HookedTransformer(cfg)
    # Biases are enabled by default
    for name, param in model.named_parameters():
        if "mlp.b_out" in name:
            param.requires_grad = False
    return model

model = get_seeded_model(seed = 993)

## Define Optimizer + Loss

In [7]:
print(train_data)
print(model(train_data))

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [1, 0],
        [1, 1],
        [1, 2],
        [2, 0],
        [2, 1],
        [2, 2]])
tensor([[[ 0.1324,  0.1346,  0.1444],
         [-0.1453,  0.5954,  0.4067]],

        [[ 0.1324,  0.1346,  0.1444],
         [ 0.0021,  0.2391,  0.1825]],

        [[ 0.1324,  0.1346,  0.1444],
         [-0.3875,  0.3479,  0.1415]],

        [[ 0.2610,  0.0929,  0.1536],
         [-0.1282,  0.5130,  0.3495]],

        [[ 0.2610,  0.0929,  0.1536],
         [ 0.0193,  0.1562,  0.1249]],

        [[ 0.2610,  0.0929,  0.1536],
         [-0.3633,  0.2177,  0.0502]],

        [[-0.0580, -0.1071, -0.0999],
         [-0.1084,  0.7052,  0.5020]],

        [[-0.0580, -0.1071, -0.0999],
         [ 0.0397,  0.3555,  0.2830]],

        [[-0.0580, -0.1071, -0.0999],
         [-0.3493,  0.5255,  0.2887]]], grad_fn=<ViewBackward0>)


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd, betas=betas)
def loss_fn(logits, labels):
    if len(logits.shape)==3:
        logits = logits[:, -1]
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()
train_logits = model(train_data)
train_loss = loss_fn(train_logits, train_labels)
print(train_loss)
test_logits = model(test_data)
test_loss = loss_fn(test_logits, test_labels)
print(test_loss)
print("Uniform loss:")
print(np.log(INPUT_DIM * INPUT_DIM))

tensor(1.1912, dtype=torch.float64, grad_fn=<NegBackward0>)
tensor(nan, dtype=torch.float64, grad_fn=<NegBackward0>)
Uniform loss:
2.1972245773362196


## Actually Train

**Weird Decision:** Training the model with full batch training rather than stochastic gradient descent. We do this so to make training smoother and reduce the number of slingshots.

In [9]:
def print_stats(model):
    # Extract the p 2-dimensional tensors, vector i is vec[:, i]
    vec = model.W_U.data

    # Function to compute the angle between two vectors
    def compute_angle(v1, v2):
        cos_theta = torch.dot(v1, v2) / (torch.norm(v1) * torch.norm(v2))
        angle = torch.acos(cos_theta) * (180.0 / np.pi)
        return angle

    # Compute pairwise angles
    # for i in range(vec.shape[1]):
    #     for j in range(i+1, vec.shape[1]):
    #         angle = compute_angle(vec[:, i], vec[:, j])
    #         print(f"Angle between {i} and {j}: {angle.item():.2f}°")
    #     print(f"Norm of vector {i}: {torch.norm(vec[:, i]):.2f}")

print_stats(model)

In [10]:
def train_model(model, train_data, train_labels, test_data, test_labels, num_epochs = num_epochs, loss_target = None):
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=lr, weight_decay=wd, betas=betas
    )
    train_losses = []
    test_losses = []
    model_checkpoints = []
    checkpoint_epochs = []
    if TRAIN_MODEL:
        for epoch in tqdm.tqdm(range(num_epochs)):
            train_logits = model(train_data)
            train_loss = loss_fn(train_logits, train_labels)
            train_loss.backward()
            train_losses.append(train_loss.item())
            if loss_target is not None and train_loss.item() < loss_target:
                print(f"Loss target {loss_target} reached with loss {train_loss.item()} at epoch {epoch}")
                break

            optimizer.step()
            optimizer.zero_grad()

            with torch.inference_mode():
                test_logits = model(test_data)
                test_loss = loss_fn(test_logits, test_labels)
                test_losses.append(test_loss.item())

            if ((epoch+1)%checkpoint_every)==0:
                checkpoint_epochs.append(epoch)
                model_checkpoints.append(copy.deepcopy(model.state_dict()))
                print_stats(model)
                print(f"Epoch {epoch} Train Loss {train_loss.item()} Test Loss {test_loss.item()}")
    if TRAIN_MODEL:
        torch.save(
            {
                "model":model.state_dict(),
                "config": model.cfg,
                "checkpoints": model_checkpoints,
                "checkpoint_epochs": checkpoint_epochs,
                "test_losses": test_losses,
                "train_losses": train_losses,
            },
            PTH_LOCATION)

train_model(
    model, train_data, train_labels, test_data, test_labels
)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 499 Train Loss 0.5963916712405357 Test Loss nan
Epoch 999 Train Loss 0.21747547169961456 Test Loss nan
Epoch 1499 Train Loss 0.008244498748145694 Test Loss nan
Epoch 1999 Train Loss 0.0026728980357629602 Test Loss nan
Epoch 2499 Train Loss 0.0012444408153213494 Test Loss nan
Epoch 2999 Train Loss 0.0006981506849781179 Test Loss nan
Epoch 3499 Train Loss 0.00043569503453585627 Test Loss nan
Epoch 3999 Train Loss 0.00029152461820886315 Test Loss nan
Epoch 4499 Train Loss 0.00019844385815571584 Test Loss nan
Epoch 4999 Train Loss 0.00013990493860122004 Test Loss nan
Epoch 5499 Train Loss 0.0001004295480539135 Test Loss nan
Epoch 5999 Train Loss 7.31394082077081e-05 Test Loss nan
Epoch 6499 Train Loss 5.3726162289120084e-05 Test Loss nan
Epoch 6999 Train Loss 4.008409974102984e-05 Test Loss nan
Epoch 7499 Train Loss 2.952947689677196e-05 Test Loss nan
Epoch 7999 Train Loss 2.2238729522731337e-05 Test Loss nan
Epoch 8499 Train Loss 1.6790130198897674e-05 Test Loss nan
Epoch 8999 Train

In [11]:
if not TRAIN_MODEL:
    cached_data = torch.load(PTH_LOCATION)
    model.load_state_dict(cached_data['model'])
    model_checkpoints = cached_data["checkpoints"]
    checkpoint_epochs = cached_data["checkpoint_epochs"]
    test_losses = cached_data['test_losses']
    train_losses = cached_data['train_losses']

# Activations

In [12]:
def create_cache(model):
    input = train_data
    print("Transposed Input:\n", input.transpose(0, 1))
    logits, cache = model.run_with_cache(input)
    print("Labels: ", train_labels)
    print("Logits of last token:\n", logits[:, -1, :])
    print("Unembed:\n", model.W_U.data)
    print("Last layer before unembed:\n", cache.cache_dict["blocks.0.hook_resid_post"][:, -1, :])
    return cache

cache = create_cache(model)

Transposed Input:
 tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 1, 2, 0, 1, 2, 0, 1, 2]])
Labels:  tensor([0, 1, 2, 1, 1, 0, 2, 2, 0])
Logits of last token:
 tensor([[ 4.6067e+00, -7.3325e+00, -5.6263e+00],
        [-1.2100e+00,  1.4056e+01, -1.1336e+00],
        [-1.1399e+00, -2.5416e+01,  1.1703e+01],
        [ 3.3368e-02,  1.1553e+01, -2.7741e+00],
        [-5.9428e+00,  3.3759e+01,  1.7660e+00],
        [ 6.2936e+00, -8.1738e+00, -8.6916e+00],
        [-1.9211e+00, -1.6715e+01,  1.0390e+01],
        [-3.8065e+00, -1.4896e+01,  1.3527e+01],
        [ 6.0160e+00, -1.3005e+01, -6.5537e+00]], grad_fn=<SliceBackward0>)
Unembed:
 tensor([[ 1.0326, -0.9932, -1.7193],
        [ 0.0391, -1.7637,  0.5023]])
Last layer before unembed:
 tensor([[  3.5075,   2.0904],
        [ -1.7789,  -7.0596],
        [ -2.5756,  15.7691],
        [ -0.6035,  -6.3025],
        [ -6.0298, -15.8371],
        [  5.1582,   1.6378],
        [ -3.1577,  11.1636],
        [ -4.9834,  11.1604],
        [  4.7776

In [13]:
# Get a list of all activations stored in the cache, especially their names
cache.cache_dict

{'hook_embed': tensor([[[ 0.4180,  0.9027],
          [ 0.4180,  0.9027]],
 
         [[ 0.4180,  0.9027],
          [-1.0277, -0.1865]],
 
         [[ 0.4180,  0.9027],
          [ 1.6651, -0.0442]],
 
         [[-1.0277, -0.1865],
          [ 0.4180,  0.9027]],
 
         [[-1.0277, -0.1865],
          [-1.0277, -0.1865]],
 
         [[-1.0277, -0.1865],
          [ 1.6651, -0.0442]],
 
         [[ 1.6651, -0.0442],
          [ 0.4180,  0.9027]],
 
         [[ 1.6651, -0.0442],
          [-1.0277, -0.1865]],
 
         [[ 1.6651, -0.0442],
          [ 1.6651, -0.0442]]]),
 'hook_pos_embed': tensor([[[ 0.3340,  1.2297],
          [-0.2243, -1.8267]],
 
         [[ 0.3340,  1.2297],
          [-0.2243, -1.8267]],
 
         [[ 0.3340,  1.2297],
          [-0.2243, -1.8267]],
 
         [[ 0.3340,  1.2297],
          [-0.2243, -1.8267]],
 
         [[ 0.3340,  1.2297],
          [-0.2243, -1.8267]],
 
         [[ 0.3340,  1.2297],
          [-0.2243, -1.8267]],
 
         [[ 0.3340,  1.

# Animations

In [14]:
from manim import *

config.media_width = "80%"
config.verbosity = "WARNING"
config.preview = False

/home/pweiss/Research/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [15]:
class VectorParams:
    def __init__(self, values = [], color = WHITE, label = ""):
        self.values = values
        self.color = color
        self.label = label
    def __repr__(self) -> str:
        return str(self.values) + "(" + str(self.color) + ")"

class Data:
    def __init__(self):
        self.vectors: list[list[VectorParams]] = [[]]
        self.steps = 0
        self.current_labels = set()

    def add_vector(self, vector, color = WHITE, label = ""):
        if label not in self.current_labels:
            self.current_labels.add(label)
            self.vectors[self.steps].append(VectorParams(values = vector, color = color, label = label))

    def next_step(self):
        self.steps += 1
        self.vectors.append([])
        self.current_labels = set()

    def add_vectors_at_hook(self, c: ActivationCache, hook: str, color0 = WHITE, color1 = WHITE, input_labels = None, input_colors = None):
        if input_labels is None:
            input_labels = ["" for i in range(c.cache_dict[hook].shape[0])]
        for i in range(c.cache_dict[hook].shape[0]):
            self.add_vector(c.cache_dict[hook][i][0].cpu(), color = color0, label = input_labels[i][:1])
            self.add_vector(
                c.cache_dict[hook][i][1].cpu(), color=color1 if input_colors is None else input_colors[i], label=input_labels[i][:2]
            )


def compile_data_vectors(cache, input_labels=None, input_colors=None):
    # Set default value as list of empty strings
    vectors = Data()
    vectors.add_vectors_at_hook(cache, "hook_embed", color1 = GRAY, input_labels=input_labels, input_colors=input_colors)
    vectors.next_step()
    vectors.add_vectors_at_hook(cache, "blocks.0.hook_resid_pre", input_labels=input_labels, input_colors=input_colors)
    vectors.next_step()
    vectors.add_vectors_at_hook(cache, "blocks.0.hook_resid_mid", color0 = GRAY, input_labels=input_labels, input_colors=input_colors)
    vectors.next_step()
    vectors.add_vectors_at_hook(cache, "blocks.0.hook_resid_post", color0 = GRAY, input_labels=input_labels, input_colors=input_colors)
    
    print(vectors.vectors)
    return vectors


vectors = compile_data_vectors(cache)

[[tensor([0.4180, 0.9027])(#FFFFFF)], [tensor([0.7520, 2.1324])(#FFFFFF)], [tensor([ 8.4941, 14.1598])(#888888)], [tensor([-16.8020,  28.7105])(#888888)]]


In [16]:
def change_font_size(labeled_arrow: LabeledArrow, new_size):
    # print(labeled_arrow, labeled_arrow.submobjects)
    # print(labeled_arrow.submobjects[-1].font_size)
    if not isinstance(labeled_arrow, LabeledArrow):
        return
    label = labeled_arrow.submobjects[-1]
    box = labeled_arrow.submobjects[-2]
    if not isinstance(box, BackgroundRectangle):
        box = labeled_arrow.submobjects[-3]
    coords = label.get_center()
    # print(new_size)
    labeled_arrow.submobjects[-1] = MathTex(
        label.get_tex_string(), color=label.color, font_size=new_size
    )
    # print("size=", labeled_arrow.submobjects[-1].font_size)
    label = labeled_arrow.submobjects[-1]
    label.move_to(coords)
    box.width = label.width + 2 * box.buff
    box.height = label.height + 2 * box.buff

In [17]:
DOT_SCALE = 0.01
class VisualizeTransformer(MovingCameraScene):
    def construct(self):
        print("v=", vectors.vectors)
        axes = Axes(
            x_range = [-20, 20, 1],
            y_range = [-20, 20, 1],
            x_axis_config={
                "numbers_to_include": np.arange(-18, 18.1, 3),
                "font_size": 24
            },
            y_axis_config={
                "numbers_to_include": np.arange(-18, 18.1, 3), 
                "font_size": 24            
            },
            x_length = 40,
            y_length = 40,
            axis_config={"color": GREEN}
        )

        scale = ValueTracker(2)

        dots = VGroup()
        def update_scale(self):
            return
            # TODO: Make the scaling nicer
            self.stroke_width = 6 * scale.get_value()
            change_font_size(self, 48 * scale.get_value())
            # print("New font size: ", self.font_size)

        # Embedding arrows
        for i, t in enumerate(vectors.vectors[0]):
            # print(t, t.numpy())
            # arrow = LabeledArrow(
            #     start=ORIGIN,
            #     end=np.append(t.values.numpy(), 0),
            #     buff = 0,
            #     label = t.label,
            #     label_frame = False,
            #     label_color=YELLOW,
            #     color = t.color,
            #     max_stroke_width_to_length_ratio = 100,
            # )

            # arrow.add_updater(update_scale)
            # arrows.add(arrow)
            dot = LabeledDot(
                point=np.append(t.values.numpy(), 0),
                label=t.label,
                color=t.color,
                radius=DOT_SCALE * (len(vectors.vectors[0]) - i) + 0.2,
            )

            dot.set_opacity(0.5)

            dot.add_updater(update_scale)
            dots.add(dot)

        # Transitioning the arrows through the model
        self.add(axes, axes.get_axis_labels(), dots)
        for step in range(1, len(vectors.vectors)):
            new_dots = VGroup()
            transition_arrows = VGroup()
            for i, t in enumerate(vectors.vectors[step]):
                # print(t, t.numpy())
                # new_arrow = LabeledArrow(
                #     start=ORIGIN,
                #     end=np.append(t.values.numpy(), 0),
                #     buff=0,
                #     label=t.label,
                #     label_frame=False,
                #     label_color=YELLOW,
                #     color=t.color,
                #     max_stroke_width_to_length_ratio=100,
                # )
                # new_arrow.add_updater(update_scale)
                # new_arrows.add(new_arrow)
                new_dot = LabeledDot(
                    point=np.append(t.values.numpy(), 0),
                    label=t.label,
                    color=t.color,
                    radius = DOT_SCALE * (len(vectors.vectors[step]) - i) + 0.2,
                )
                new_dot.set_opacity(0.5)
                new_dot.add_updater(update_scale)
                new_dots.add(new_dot)

                transition_arrow = Arrow(
                    start=dots[i].arc_center,
                    end=new_dots[i].arc_center,
                    buff=0,
                    color=RED,
                )
                transition_arrow.add_updater(update_scale)
                transition_arrows.add(transition_arrow)

            view = SurroundingRectangle(new_dots)
            factor = max(
                view.width / self.camera.frame_width,
                view.height / self.camera.frame_height,
            )
            print(
                factor,
                self.camera.frame_width, view.width,
                self.camera.frame_height, view.height,
            )
            self.wait()
            self.play(FadeIn(transition_arrows), self.camera.auto_zoom(view, margin = 2), scale.animate.set_value(scale.get_value() * factor))
            self.wait()
            self.play(
                ReplacementTransform(dots, new_dots)
            )
            self.wait()
            self.play(FadeOut(transition_arrows))
            self.wait()
            dots = new_dots

        # Unembedding Arrows
        embedding_arrows = VGroup()
        data = model.W_U.data
        print("unembed: ", data)
        for i in range(data.size()[1]):
            embedding_arrow = LabeledArrow(
                start=ORIGIN,
                end=[data[0, i].item(), data[1, i].item(), 0],
                label=str(i),
                color=LIGHT_PINK,
                buff=0,
                max_stroke_width_to_length_ratio=100,
            )
            embedding_arrows.add(embedding_arrow)
        self.play(FadeIn(embedding_arrows))
        self.wait()

# v = VisualizeTransformer()
# v.construct()

In [18]:
%env CUDA_LAUNCH_BLOCKING=1
%env TORCH_USE_CUDA_DSA=1

env: CUDA_LAUNCH_BLOCKING=1
env: TORCH_USE_CUDA_DSA=1


In [19]:
%%manim -qh Video

import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'


input_dim = 4
output_dim = 2
train_data, train_labels, test_data, test_labels = get_training_data(input_dim, output_dim, data_seed=999)
model = get_seeded_model(999, input_dim, output_dim)
train_model(model, train_data, train_labels, test_data, test_labels, num_epochs = 20000, loss_target = 1/(input_dim ** 2 * 4))
cache = create_cache(model)
arrow_labels = ["".join([str(d.item()) for d in v]) for v in train_data]
colors = [BLUE, YELLOW, GREEN, RED]
arrow_colors = [colors[l] for l in train_labels]
vectors = compile_data_vectors(cache, input_labels=arrow_labels, input_colors = arrow_colors)
print("Labels: ", arrow_labels)

class Video(VisualizeTransformer):
    def construct(self):
        VisualizeTransformer.construct(self)

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [1, 0],
        [1, 1],
        [1, 2],
        [1, 3],
        [2, 0],
        [2, 1],
        [2, 2],
        [2, 3],
        [3, 0],
        [3, 1],
        [3, 2],
        [3, 3]])
tensor([0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0])
torch.Size([16, 2])
tensor([], size=(0, 2), dtype=torch.int64)
tensor([], dtype=torch.int64)
torch.Size([0, 2])


  0%|          | 0/20000 [00:00<?, ?it/s]

Epoch 499 Train Loss 0.41370474859541106 Test Loss nan
Epoch 999 Train Loss 0.21454377600874963 Test Loss nan
Epoch 1499 Train Loss 0.08929511159139729 Test Loss nan
Epoch 1999 Train Loss 0.0873764823601811 Test Loss nan
Epoch 2499 Train Loss 0.08700525877129292 Test Loss nan
Epoch 2999 Train Loss 0.08686360368017351 Test Loss nan
Epoch 3499 Train Loss 0.08679235683343546 Test Loss nan
Epoch 3999 Train Loss 0.08675086852883909 Test Loss nan
Epoch 4499 Train Loss 0.08672390230506992 Test Loss nan
Epoch 4999 Train Loss 0.08670552669529044 Test Loss nan
Epoch 5499 Train Loss 0.08669331189140848 Test Loss nan
Epoch 5999 Train Loss 0.08668164177630497 Test Loss nan
Epoch 6499 Train Loss 0.08667355192831118 Test Loss nan
Epoch 6999 Train Loss 0.08666749664564917 Test Loss nan
Epoch 7499 Train Loss 0.0866627468616861 Test Loss nan
Epoch 7999 Train Loss 0.08665908031492568 Test Loss nan
Epoch 8499 Train Loss 0.08665586884163146 Test Loss nan
Epoch 8999 Train Loss 0.08665372916823677 Test Loss 

Manim Community v0.19.0

v= [[tensor([-0.7126, -1.4422])(#FFFFFF), tensor([-0.7126, -1.4422])(#58C4DD), tensor([ 0.4653, -0.5544])(#58C4DD), tensor([1.7884, 2.3283])(#FFFF00), tensor([1.4434, 1.9512])(#FFFF00), tensor([ 0.4653, -0.5544])(#FFFFFF), tensor([-0.7126, -1.4422])(#58C4DD), tensor([ 0.4653, -0.5544])(#FFFF00), tensor([1.7884, 2.3283])(#FFFF00), tensor([1.4434, 1.9512])(#FFFF00), tensor([1.7884, 2.3283])(#FFFFFF), tensor([-0.7126, -1.4422])(#FFFF00), tensor([ 0.4653, -0.5544])(#FFFF00), tensor([1.7884, 2.3283])(#58C4DD), tensor([1.4434, 1.9512])(#FFFF00), tensor([1.4434, 1.9512])(#FFFFFF), tensor([-0.7126, -1.4422])(#FFFF00), tensor([ 0.4653, -0.5544])(#58C4DD), tensor([1.7884, 2.3283])(#58C4DD), tensor([1.4434, 1.9512])(#58C4DD)], [tensor([-1.4972, -0.7341])(#FFFFFF), tensor([ 0.3471, -1.7870])(#58C4DD), tensor([ 1.5250, -0.8991])(#58C4DD), tensor([2.8482, 1.9835])(#FFFF00), tensor([2.5031, 1.6065])(#FFFF00), tensor([-0.3193,  0.1537])(#FFFFFF), tensor([ 0.3471, -1.7870])(#58C4DD), tensor([ 1.5250, -

2.885034153373022 13.712465523613822 8.422848825454711 7.713261857032776 22.253023891448976


0.6765938607331213 43.11648691813151 15.326490745544433 24.253023891448976 16.409447069168092


unembed:  tensor([[-1.4806,  2.1872],
        [ 1.3179, -0.9218]])
